Cuaderno de jupyter para recuperar las IDs internas de las normas contenidas en los servicios de Ley Chile, servicio de la Biblioteca del Congreso Nacional de Chile. Se utiliza una petición HTTP para recuperar las IDs de normas paginadas en la web del Ley Chile.

In [14]:
!python3 -m pip install requests


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [15]:
import requests
import time
import csv

In [16]:
base_url = "https://nuevo.leychile.cl/servicios/buscarjson?itemsporpagina=50&npagina={}&tipoviene=1&fc_de=&fc_ra=&seleccionado=0&fc_rp=&totalitems=385002&orden=2&fc_pb=&fc_pr=&exacta=0&cadena=&fc_tn="

# Número total de páginas segun la web de Ley Chile al momento del desarrollo del cuaderno.
total_paginas = 7701

# Número de página inicial
pagina_inicial = 1

# ID incremental utilizada de forma interna para la recuperación de todas las IDs del BCN
id_interna = 1

In [33]:
# Leer el archivo de progreso si existe, este archivo es una estructura de control ante fallas en la ejecución del cuaderno.
try:
    with open('progreso.txt', 'r') as f:
        pagina_inicial = int(f.readline().strip()) + 1
except FileNotFoundError:
    pass

In [34]:
# Si el archivo ids.csv existe, determinar el último id registrado. Esto es para continuar con la recuperación de ids.
try:
    with open('ids.csv', 'r') as f:
        for row in csv.reader(f):
            id_interna = int(row[0]) + 1
except FileNotFoundError:
    pass

In [ ]:
# Recorrer todas las páginas, desde la página inicial hasta la última página según la web de Ley Chile. Por cada página se realiza una petición HTTP y se recuperan las IDs de las normas.
for pagina in range(pagina_inicial, total_paginas + 1):
    # Crear la URL para la página actual
    url = base_url.format(pagina)
    
    try:
        # Realizar la petición
        response = requests.get(url)
        data = response.json()
        
        # Guardar idNorma en el archivo CSV
        with open('ids.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            for norma in data[0]:
                writer.writerow([id_interna, norma['IDNORMA']])
                id_interna += 1
        
        # Guardar el progreso en el archivo de progreso
        with open('progreso.txt', 'w') as f:
            f.write(str(pagina))
        
        # Imprimir el porcentaje de progreso
        progress = (pagina / total_paginas) * 100
        print(f"Progreso: {progress:.2f}%")
    
    except requests.RequestException:
        with open('errores.txt', 'a') as f:
            f.write(f"{pagina}\n")
        print(f"Error en la página {pagina}")
    except ValueError:
        with open('errores.txt', 'a') as f:
            f.write(f"{pagina}\n")
        print(f"Error decodificando JSON en la página {pagina}")

    # Esperar 1 segundo entre peticiones para evitar alcanzar límites de tasa del servidor de BCN
    time.sleep(1)


Progreso: 90.69%
Progreso: 90.70%
Progreso: 90.72%
Progreso: 90.73%
Progreso: 90.74%
Progreso: 90.75%
Progreso: 90.77%
Progreso: 90.78%
Progreso: 90.79%
Progreso: 90.81%
Progreso: 90.82%
Progreso: 90.83%
Progreso: 90.85%
Progreso: 90.86%
Progreso: 90.87%
Progreso: 90.88%
Progreso: 90.90%
Progreso: 90.91%
Progreso: 90.92%
Progreso: 90.94%
Progreso: 90.95%
Progreso: 90.96%
Progreso: 90.98%
Progreso: 90.99%
Progreso: 91.00%
Progreso: 91.01%
Progreso: 91.03%
Progreso: 91.04%
Progreso: 91.05%
Progreso: 91.07%
Progreso: 91.08%
Progreso: 91.09%
Progreso: 91.11%
Progreso: 91.12%
Progreso: 91.13%
Progreso: 91.14%
Progreso: 91.16%
Progreso: 91.17%
Progreso: 91.18%
Progreso: 91.20%
Progreso: 91.21%
Progreso: 91.22%
Progreso: 91.23%
Progreso: 91.25%
Progreso: 91.26%
Progreso: 91.27%
Progreso: 91.29%
Progreso: 91.30%
Progreso: 91.31%
Progreso: 91.33%
Progreso: 91.34%
Progreso: 91.35%
Progreso: 91.36%
Progreso: 91.38%
Progreso: 91.39%
Progreso: 91.40%
Progreso: 91.42%
Progreso: 91.43%
Progreso: 91.4

In [37]:
# Leer las páginas con errores desde el archivo errors.txt
with open("errores.txt", "r") as file:
    error_pages = file.readlines()
    error_pages = [int(page.strip()) for page in error_pages]

# Estructura para guardar páginas que aún dan error
new_error_pages = []


In [38]:
# Obtener el último ID interno registrado
with open("ids.csv", "r") as file:
    lines = file.readlines()
    if lines:
        last_id = int(lines[-1].split(",")[0])
    else:
        last_id = 0

In [40]:
# Recorrer las páginas con errores y recuperar las IDs de las normas de cada una de ellas y guardarlas en el archivo ids.csv.
for page in error_pages:
    url = base_url.format(page)
    
    try:
        # Realizar la petición
        response = requests.get(url)
        data = response.json()
        
        # Abrir el archivo CSV en modo append para agregar nuevas IDs
        with open("ids.csv", "a", newline='') as file:
            writer = csv.writer(file)
            
            for norma in data[0]:
                last_id += 1
                writer.writerow([last_id, norma["IDNORMA"]])

        print(f"Página {page} procesada con éxito.")
    
    except requests.RequestException as e:
        new_error_pages.append(page)
        print(f"Error en la página {page}: {e}")
    except ValueError:
        new_error_pages.append(page)
        print(f"Error decodificando JSON en la página {page}. Respuesta:\n{response.text}")

    # Esperar 1 segundo entre peticiones para evitar alcanzar límites de tasa
    time.sleep(1)

Página 287 procesada con éxito.
Página 802 procesada con éxito.
Página 874 procesada con éxito.
Página 1772 procesada con éxito.
Página 1799 procesada con éxito.
Página 1800 procesada con éxito.
Página 1801 procesada con éxito.
Página 2316 procesada con éxito.
Página 2379 procesada con éxito.
Página 2463 procesada con éxito.
Página 2600 procesada con éxito.
Página 2625 procesada con éxito.
Página 2627 procesada con éxito.
Página 2675 procesada con éxito.
Página 3117 procesada con éxito.
Página 3961 procesada con éxito.
Página 4181 procesada con éxito.
Página 5160 procesada con éxito.
Página 5190 procesada con éxito.
Página 5451 procesada con éxito.
Página 5863 procesada con éxito.
Página 6113 procesada con éxito.
Página 6230 procesada con éxito.
Página 6406 procesada con éxito.
Página 6791 procesada con éxito.
Página 6863 procesada con éxito.
Página 6914 procesada con éxito.
Página 7078 procesada con éxito.
Página 7487 procesada con éxito.


In [41]:
# Guardar páginas que aún dan error, en caso de que existan. De ser así se pueden volver a ejecutar las peticiones para recuperar las IDs.
with open("errores.txt", "w") as file:
    for page in new_error_pages:
        file.write(f"{page}\n")